In [ ]:
# Show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Imports.
import os

projectdir = '/Users/michielkleinnijenhuis/PMCdata/1.projects/FGS'
dataset = 'FGS_RL1_Exp002'

datadir = os.path.join(projectdir, dataset)

os.makedirs(datadir, exist_ok=True)
os.chdir(datadir)
f'working in directory: {os.path.abspath(".")}'
parameter_file = f'{dataset}.yml'
image_in = os.path.join(datadir, 'blocks')
image_in = f'{datadir}/' + '{f}.czi'


In [ ]:
# NOTE: lowres => get info from blockfile, not czi
blockdir = os.path.join(datadir, 'blocks')
image_in = f'{blockdir}/' + '{f}.h5'

from stapl3d.segmentation import segment
segment3r = segment.Segment3r(image_in, parameter_file, prefix=dataset, step_id='segmentation_norm', datadir=datadir)
segment3r.estimate()


In [ ]:
# NOTE: lowres => get info from blockfile, not czi
blockdir = os.path.join(datadir, 'blocks')
image_in = f'{blockdir}/' + '{f}.h5'

from stapl3d.deep_learning import unet
torch3r = unet.Torch3r(image_in, parameter_file)
torch3r.train()


In [ ]:
torch3r.view([5], images=['image_lr', 'image_lr_norm', 'memb/prob'], labels=['mask_lr', 'memb/pred'])


```
# ssh -i ~/.ssh/id_rsa_hpc -X mkleinnijenhuis@hpcs03.op.umcutrecht.nl
# srun -c 1 --time 04:00:00 --mem 100G --pty bash
# srun -c 4 --partition=gpu --gpus-per-node=RTX6000:1 --time 04:00:00 --mem 10G --pty bash
# srun --partition=gpu --time 01:00:00 --mem 10G --pty bash

source "${HOME}/.stapl3d.ini" && load_stapl3d_config

projectdir='/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS'
dataset='FGS_RL1_Exp002'
load_dataset "${projectdir}" "${dataset}"

parfile="${dataset}.yml"
#[[ -f "${parfile}" ]] || init_dataset
load_parameters "${dataset}" -v "${parfile}"


unset blockstems
blockstems=()
for filename in `ls $datadir/blocks`; do
    dstem="${filename%.*}"
    blockstems+=( "$dstem" )
done
echo "${#blockstems[@]}"

dataset__alias='FGS002'
image_in='blocks/{f}.h5'
jid=h
submit $( generate_script segmentation estimate) $jid

# NOTE: in go_<>.py
segment3r = segment.Segment3r('blocks/{f}.h5', "FGS_RL1_Exp002.yml", datadir='/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002')
# NOTE: in go_<>.sh
#SBATCH --array=1-81:1
```


In [ ]:
function get_py_torch {
    echo "from stapl3d import stapl3d_torch"
    echo "torch3r = stapl3d_torch.Torch3r(image_in, parameter_file)"
}
function get_py_torch_train {
    get_py_header
    get_py_torch
    echo "torch3r.${step}()"
}
function get_cmd_torch_train { get_cmd "" ; }


In [ ]:



dataset__alias='FGS002'
image_in='blocks/{f}.h5'
jid=
submit $( generate_script torch train) $jid


In [ ]:
#!/usr/bin/env bash

#SBATCH --job-name=go_FGS002_torch_train
#SBATCH --partition=gpu
#SBATCH --gpus-per-node=RTX6000:1
#SBATCH --cpus-per-task=4
#SBATCH --mem=10G
#SBATCH --time=01:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --array=1-1:1
#SBATCH --output=/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002/go_FGS002_torch_train.sh.o%A.%a
#SBATCH --error=/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002/go_FGS002_torch_train.sh.e%A.%a

TASK_ID=${SLURM_ARRAY_TASK_ID}
idx=$((TASK_ID - 1))


source /hpc/local/CentOS7/pmc_rios/projects/STAPL3D/stapl3d/pipelines/functions.sh
load_dataset /hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS FGS_RL1_Exp002
load_parameters FGS_RL1_Exp002 -v

source /hpc/local/CentOS7/pmc_rios/anaconda3/etc/profile.d/conda.sh
conda activate stapl3d_torch

python /hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002/go_FGS002_torch_train.py blocks/{f}.h5 FGS_RL1_Exp002.yml ${idx}
conda deactivate


In [ ]:
from stapl3d.segmentation import segment
datadir = '/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002'
image_in = 'blocks/{f}.h5'
parameter_file = 'FGS_RL1_Exp002.yml'
segment3r = segment.Segment3r(image_in, parameter_file, datadir=datadir)
segment3r.estimate()


In [ ]:
#!/usr/bin/env python

import sys
image_in = sys.argv[1]
parameter_file = sys.argv[2]
if len(sys.argv) > 3:
    idx = int(sys.argv[3])

from stapl3d.deep_learning import unet
datadir = '/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002'
image_in = 'blocks/{f}.h5'
parameter_file = 'FGS_RL1_Exp002.yml'
torch3r = unet.Torch3r(image_in, parameter_file, datadir=datadir)
torch3r.train()


In [ ]:
from stapl3d.segmentation import segment
segment3r = segment.Segment3r('blocks/{f}.h5', "FGS_RL1_Exp002.yml", datadir='/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002')
# 0 is broken
for idx in range(2, 81):
    segment3r.estimate(blocks=[idx])
# 6 is broken
for idx in range(7, 81):
    segment3r.estimate(blocks=[idx])


In [ ]:
from stapl3d import stapl3d_torch
torch3r = stapl3d_torch.Torch3r('blocks/{f}.h5', "FGS_RL1_Exp002.yml", datadir='/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002')
torch3r.train()


In [ ]:
import tracemalloc
tracemalloc.start()

from stapl3d import stapl3d_torch
torch3r = stapl3d_torch.Torch3r('blocks/{f}.h5', "FGS_RL1_Exp002.yml", datadir='/hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002')
torch3r.train()

# show how much RAM the above code allocated and the peak usage
current, peak =  tracemalloc.get_traced_memory()
print(f"{current/1024**2:0.0f}, {peak/1024**2:0.0f}")
tracemalloc.stop()


In [ ]:
for filename in `ls *.tar.bz2`; do
    tar -xf $filename
done


In [ ]:
import numpy as np
shape = (4, 3, 265, 265)
c = np.zeros(shape)
b = np.reshape(np.moveaxis(c, 0, 3), list(shape[1:3]) + [-1])
b.shape


In [ ]:
conda activate stapl3d_torch
tensorboard --port 6006 --host 0.0.0.0 --logdir /hpc/pmc_rios/mkleinnijenhuis/1.projects/FGS/FGS_RL1_Exp002/logs/tb &
alias sg='squeue | grep gpu'

ssh -L6006:hpcs03.compute.hpc:6006 -X mkleinnijenhuis@hpcs03.op.umcutrecht.nl
localhost:6006


In [ ]:
(stapl3d_torch) [mkleinnijenhuis@n0100 FGS_RL1_Exp002]$ python -m torch.utils.collect_env
Collecting environment information...
PyTorch version: 1.12.1
Is debug build: False
CUDA used to build PyTorch: Could not collect
ROCM used to build PyTorch: N/A

OS: CentOS Linux release 7.9.2009 (Core) (x86_64)
GCC version: (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Clang version: Could not collect
CMake version: version 2.8.12.2
Libc version: glibc-2.17

Python version: 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21)  [GCC 10.3.0] (64-bit runtime)
Python platform: Linux-3.10.0-1160.25.1.el7.x86_64-x86_64-with-glibc2.17
Is CUDA available: False
CUDA runtime version: Could not collect
GPU models and configuration: GPU 0: Quadro RTX 6000
Nvidia driver version: 525.60.13
cuDNN version: Probably one of the following:
/usr/lib64/libcudnn.so.8.3.3
/usr/lib64/libcudnn_adv_infer.so.8.3.3
/usr/lib64/libcudnn_adv_train.so.8.3.3
/usr/lib64/libcudnn_cnn_infer.so.8.3.3
/usr/lib64/libcudnn_cnn_train.so.8.3.3
/usr/lib64/libcudnn_ops_infer.so.8.3.3
/usr/lib64/libcudnn_ops_train.so.8.3.3
HIP runtime version: N/A
MIOpen runtime version: N/A
Is XNNPACK available: True

Versions of relevant libraries:
[pip3] numpy==1.21.0
[pip3] numpydoc==1.5.0
[pip3] torch==1.12.1
[pip3] torchvision==0.13.0a0+8069656
[conda] mkl                       2022.1.0           h84fe81f_915    conda-forge
[conda] numpy                     1.21.0           py39hdbf815f_0    conda-forge
[conda] numpydoc                  1.5.0              pyhd8ed1ab_0    conda-forge
[conda] pytorch                   1.12.1          cpu_py39h3439074_1    conda-forge
[conda] torchvision               0.13.0          cpu_py39hedfcc36_0    conda-forge


In [ ]:
import torch
index = torch.tensor([2])
index
index.cpu().numpy()
